# Aula 5: SQL - Exercícios

### Case Olist

A Olist é uma loja de departamentos que participa dos principais marketplaces do país:  Mercado Livre, Walmart.com, Americanas.com, Submarino, Casas Bahia entre outros. Mais informações podem ser encontradas [aqui](https://blog.olist.com/olist-chega-para-facilitar-a-vida-de-quem-quer-vender-na-internet-e-em-grandes-varejistas/).

![](https://ensinandomaquinasblog.files.wordpress.com/2020/11/f1.png?w=1024)
![](https://i.imgur.com/Jory0O3.png)

Abaixo, segue uma descrição de cada uma das tabelas contidas no arquivo `salesOlist.db` e em qual arquivo as informações se encontram. Também podem ser conferidas as informações sobre cada coluna.
 - **Customers**: guarda a cidade, o CEP e o estado do consumidor. Chave: customer_id.
  - Colunas:
    - **customer_id:** cada pedido possui um único `customer_id`;
    - **customer_unique_id:** identificador único de cada consumidor;
    - **customer_zip_code_prefix:** primeiros 5 dígitos do CEP do consumidor;
    - **customer_city:** cidade onde o consumidor se encontra; 
    - **customer_state:** estado onde o consumidor se encontra.
 - **Geolocation**: guarda informações de cidades tais como o CEP, latitude, longitude, nome e estado;
  - Colunas:
    - **geolocation_zip_code_prefix:** 5 primeiros dígitos do local;
    - **geolocation_lat:** latitude do local;
    - **geolocation_lng:** longitude do local;
    - **geolocation_city:** cidade onde se encontra o local; e
    - **geolocation_state:** estado onde se encontra o local.
 - **OrderItems**: guarda informações sobre quais produtos foram vendidos em cada pedido, seu valor, seu vendedor, preço de venda e preço de frete.
  - Colunas:
    - **order_id:** identificador único dos pedidos;
    - **order_item_id:** identificador sequencial indicando a quantidade de produtos por pedido;
    - **product_id:** identificador único de cada produto;
    - **seller_id:** identificador único de cada vendedor;
    - **price:** preço cobrado por produto; e
    - **freight_value:** preço cobrado por frete.
 - **OrderPayments**: guarda informações sobre o pagamento, parcelas e o valor pago por cada pedido.
  - Colunas:
    - **order_id:** identificador único de cada pedido;
    - **payment_sequential**;
    - **payment_type:** método escolhido para o pagamento;
    - **payment_installments:** número de parcelas; e
    - **payment_value:** total pago.
 - **OrderReviews**: guarda informações sobre *reviews* do pedido.
  - Colunas:
    - **review_id:** identificador único de cada avaliação;
    - **order_id:** identificador único de cada pedido;
    - **review_score:** nota dada ao pedido;
    - **review_comment_title:** título do comentário;
    - **review_comment_message:** comentário sobre o pedido;
    - **review_creation_date:** data do envio do questionário de satisfação; e
    - **review_answer_timestamp:** data da resposta ao questionário de satisfação .
 - **Orders**: guarda informações sobre o qual o consumidor efetuou a compra, o *status* do pedido, data da compra, datas de entrega (estimada e efetiva)
  - Colunas:
    - **order_id:** identificador único de cada pedido;
    - **customer_id:**; 
    - **order_status:** chave para a tabela `Customers` (cada pedido tem uma única chave);
    - **order_purchase_timestamp:** *timestamp* para a data do pedido;
    - **order_approved_at:** *timestamp* para a aprovação do pagamento;
    - **order_delivered_carrier_date:** *timestamp* para a data de postagem do pedido;
    - **order_delivered_customer_date:** data da entrega do pedido; e
    - **order_estimated_delivery_date:** data estimada da entrega.
 - **Products**: guarda informações sobre sobre a categoria e tamanho dos produtos vendidos. 
  - Colunas:
    - **product_id:** identificador único de cada pedido;
    - **product_category_name:** nome da categoria do produto;
    - **product_name_lenght:** comprimento do nome do produto;
    - **product_description_lenght:** comprimento da descrição do produto;
    - **product_photos_qty:** quantidade de fotos do produto;
    - **product_weight_g:** massa do produto em gramas;
    - **product_length_cm:** comprimento do produto em cm;
    - **product_height_cm:** altu e
    - product_width_cm.
 - **Sellers**: guarda informações do vendedor, tais como a cidade, estado e o código postal
  - Colunas:
    -**seller_id:** identificador único do vendedor;
    - **seller_zip_code_prefix:** guarda informações dos 4 primeiros dígitos do CEP do vendedor;
    - **seller_city:** cidade do vendedor; e
    - **seller_state:** estado do vendedor.
 - **QualifiedLeads**: informações de *leads* da Olist.
  - Colunas:
    - **mql_id:** identificador único da *lead*;
    - **first_contact_date:** data do primeiro contato;
    - **landing_page_id:** página que adquiriu a *lead*; e 
    - **origin:** tipo da mídia que adquriu a *lead*.
 - **ClosedDeals**: informações de negócios com vendedores 
    - **mql_id:** identificador único da lead;
    - **seller_id:** identificador único do vendedor;
    - **sdr_id:** identificador único do representante de vendas e desenvolvimento;
    - **sr_id:** identificador único do representante de vendas;
    - **won_date:** data do fechamento do contrato;
    - **business_segment:** segmento do *lead*; 
    - **lead_type:** tipo da *lead* (*online*, *offline*, indústria, etc);
    - **lead_behaviour_profile:** 
    - **has_company:**: o *lead* tem uma companhia (documentada)?
    - **has_gtin:** a companhia possi código de barras internacional para seus produtos?
    - **average_stock:** tamanho do estoque declarado;
    - **business_type:** tipo de negócio (revendedor/fabricante/etc);
    - **declared_product_catalog_size:** tamanho do catálogo declarado; e
    - **declared_monthly_revenue:** renda mensal declarada.

In [2]:
import sqlalchemy
import pandas as pd
import sqlite3
from pandasql import sqldf

In [3]:
con = sqlite3.connect('salesOlist.db')
cursor = con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type="table" ORDER BY name')
cursor.fetchall()

[('ClosedDeals',),
 ('Customers',),
 ('Geolocation',),
 ('OrderItems',),
 ('OrderPayments',),
 ('OrderReviews',),
 ('Orders',),
 ('Products',),
 ('QualifiedLeads',),
 ('Sellers',)]

### Crie uma tabela com cada vendedor e o somatório que cada vendedor realizou (utilize a tabela `orderitems`). Em seguida, use junte essa tabela com a tabela de vendedores e ordene de forma decrescente pelo somatório que cada vendedor realizou

In [17]:
query = '''
SELECT *
FROM Sellers AS a
JOIN (
    SELECT seller_id
            ,SUM(price) AS valor_vendido_total
    FROM OrderItems
    GROUP BY seller_id
) AS b
    ON a.seller_id = b.seller_id
ORDER BY valor_vendido_total DESC
'''
pd.read_sql(query, con)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_id,valor_vendido_total
0,4869f7a5dfa277a7dca6462dcf3b52b2,14840,guariba,SP,4869f7a5dfa277a7dca6462dcf3b52b2,229472.63
1,53243585a1d6dc2643021fd1853d8905,42738,lauro de freitas,BA,53243585a1d6dc2643021fd1853d8905,222776.05
2,4a3ca9315b744ce9f8e9374361493884,14940,ibitinga,SP,4a3ca9315b744ce9f8e9374361493884,200472.92
3,fa1c13f2614d7b5c4749cbc52fecda94,13170,sumare,SP,fa1c13f2614d7b5c4749cbc52fecda94,194042.03
4,7c67e1448b00f6e969d365cea6b010ab,08577,itaquaquecetuba,SP,7c67e1448b00f6e969d365cea6b010ab,187923.89
...,...,...,...,...,...,...
3090,34aefe746cd81b7f3b23253ea28bef39,81210,curitiba,PR,34aefe746cd81b7f3b23253ea28bef39,8.00
3091,702835e4b785b67a084280efca355756,36046,juiz de fora,MG,702835e4b785b67a084280efca355756,7.60
3092,1fa2d3def6adfa70e58c276bb64fe5bb,04106,sao paulo,SP,1fa2d3def6adfa70e58c276bb64fe5bb,6.90
3093,77128dec4bec4878c37ab7d6169d6f26,02610,sao paulo,SP,77128dec4bec4878c37ab7d6169d6f26,6.50


### Crie uma tabela com cada vendedor e a média que cada vendedor realizou (utilize a tabela `orderitems`) e filtre somente os vendedores que obtiveram média acima de R\$1000. Em seguida, use junte essa tabela com a tabela de vendedores e ordene de forma decrescente pela média que cada vendedor realizou

### Avalie o Preco médio dos produtos e contagem de order_ids por Order_status e payment_type (realize um agrupamento por order status e payment type e veja qual é a forma de pagamento que as pessoas mais utilizam)

### Veja quais são as categorias mais bem avaliadas pelos clientes (crie uma tabela com o nome da categoria do produto, a contagem de product_ids e a média de review_score e ordene a tabela dos mais bem avaliados na média para os piores avaliados)